# 如何等待用户输入（函数式API）

!!!信息“先决条件”
本指南假设您熟悉以下内容：

- 使用[中断](../../concepts/ human_in_the_loop/#interrupt) 实施[人机循环](../../concepts/ human_in_the_loop) 工作流程
- [如何使用功能 API 创建 ReAct 代理](../../how-tos/react-agent-from-scratch-function)

**人在环（HIL）**交互对于[代理系统]（../../concepts/agentic_concepts/# human-in-the-loop）至关重要。等待人工输入是一种常见的 HIL 交互模式，允许代理询问用户澄清问题并在继续之前等待输入。

我们可以使用 [interrupt()](/langgraphjs/reference/functions/langgraph.interrupt-1.html) 函数在 LangGraph 中实现这一点。`interrupt` 允许我们停止图形执行以收集用户的输入，并使用收集的输入继续执行。

本指南演示了如何使用 LangGraph 的 [Functional API](../../concepts/featured_api) 实现人机交互工作流程。具体来说，我们将演示：

1. [简单使用示例](#simple-usage)
2. [如何使用ReAct代理](#agent)

## 设置

!!!注意兼容性

本指南需要 `@langchain/langgraph>=0.2.42`。

首先，安装本示例所需的依赖项：
```bash
npm install @langchain/langgraph @langchain/openai @langchain/core zod
```
接下来，我们需要为 OpenAI 设置 API 密钥（我们将使用的 LLM）：
```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```
!!!提示“为 LangGraph 开发设置 [LangSmith](https://smith.langchain.com)”

注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许您使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序 — 在[此处](https://docs.smith.langchain.com)了解有关如何开始的更多信息

## 简单用法

让我们演示一个简单的使用示例。我们将创建三个[任务](../../concepts/featured_api/#task)：

1. 追加`"bar"`。
2. 暂停以等待人工输入。恢复时，附加人工输入。
3. 追加[[代码1]]。

In [1]:
import { task, interrupt } from "@langchain/langgraph";

const step1 = task("step1", async (inputQuery: string) => {
  return `${inputQuery} bar`;
});

const humanFeedback = task(
  "humanFeedback",
  async (inputQuery: string) => {
    const feedback = interrupt(`Please provide feedback: ${inputQuery}`);
    return `${inputQuery} ${feedback}`;
  });

const step3 = task("step3", async (inputQuery: string) => {
  return `${inputQuery} qux`;
});

我们现在可以在一个简单的[entrypoint](../../concepts/featured_api/#entrypoint)中组合这些任务：

In [2]:
import { MemorySaver, entrypoint } from "@langchain/langgraph";

const checkpointer = new MemorySaver();

const graph = entrypoint({
  name: "graph",
  checkpointer,
}, async (inputQuery: string) => {
  const result1 = await step1(inputQuery);
  const result2 = await humanFeedback(result1);
  const result3 = await step3(result2);
  return result3;
});

为了实现人机循环工作流程，我们所做的就是在任务内调用 [interrupt()](../../concepts/ human_in_the_loop/#interrupt) 。

!!!提示

先前任务的结果 - 在本例中为 `step1 -- are persisted, so that they are not run again following the `interrupt`。


让我们发送一个查询字符串：

In [3]:
const config = {
  configurable: {
    thread_id: "1"
  }
};

const stream = await graph.stream("foo", config);

for await (const event of stream) {
  console.log(event);
}

{ step1: 'foo bar' }
{
  __interrupt__: [
    {
      value: 'Please provide feedback: foo bar',
      when: 'during',
      resumable: true,
      ns: [Array]
    }
  ]
}


请注意，我们在 `step1` 之后以 `interrupt` 暂停。中断提供恢复运行的指令。为了恢复，我们发出一个 [Command](../../concepts/ human_in_the_loop/#the-command-primitive) ，其中包含 `humanFeedback` 任务所需的数据。

In [4]:
import { Command } from "@langchain/langgraph";

const resumeStream = await graph.stream(new Command({
  resume: "baz"
}), config);

// 继续执行
for await (const event of resumeStream) {
  if (event.__metadata__?.cached) {
    continue;
  }
  console.log(event);
}

{ humanFeedback: 'foo bar baz' }
{ step3: 'foo bar baz qux' }
{ graph: 'foo bar baz qux' }


恢复后，运行将继续执行剩余步骤并按预期终止。

## 代理人

我们将基于[如何使用功能 API 创建 ReAct 代理](../../how-tos/react-agent-from-scratch-function) 指南中创建的代理进行构建。

在这里，我们将扩展代理，允许它在需要时向人类寻求帮助。

### 定义模型和工具

让我们首先定义我们将用于示例的工具和模型。正如在 [ReAct 代理指南](../../how-tos/react-agent-from-scratch-function) 中一样，我们将使用一个占位符工具来获取某个位置的天气描述。

在本示例中，我们将使用 [OpenAI](https://js.langchain.com/docs/integrations/providers/openai/) 聊天模型，但任何[支持工具调用](https://js.langchain.com/docs/integrations/chat/) 的模型就足够了。

In [5]:
import { ChatOpenAI } from "@langchain/openai";
import { tool } from "@langchain/core/tools";
import { z } from "zod";

const model = new ChatOpenAI({
  model: "gpt-4o-mini",
});

const getWeather = tool(async ({ location }) => {
  // 这是实际实现的占位符
  const lowercaseLocation = location.toLowerCase();
  if (lowercaseLocation.includes("sf") || lowercaseLocation.includes("san francisco")) {
    return "It's sunny!";
  } else if (lowercaseLocation.includes("boston")) {
    return "It's rainy!";
  } else {
    return `I am not sure what the weather is in ${location}`;
  }
}, {
  name: "getWeather",
  schema: z.object({
    location: z.string().describe("Location to get the weather for"),
  }),
  description: "Call to get the weather from a specific location.",
});

要向人类寻求帮助，我们可以简单地添加一个调用 [interrupt](../../concepts/ human_in_the_loop/#interrupt) 的工具：

In [6]:
import { interrupt } from "@langchain/langgraph";
import { z } from "zod";

const humanAssistance = tool(async ({ query }) => {
  const humanResponse = interrupt({ query });
  return humanResponse.data;
}, {
  name: "humanAssistance",
  description: "Request assistance from a human.",
  schema: z.object({
    query: z.string().describe("Human readable question for the human")
  })
});

const tools = [getWeather, humanAssistance];

### 定义任务

我们的任务在其他方面与 [ReAct 代理指南](../../how-tos/react-agent-from-scratch-function) 没有变化：

1. **调用模型**：我们想要使用消息列表查询我们的聊天模型。
2. **调用工具**：如果我们的模型生成工具调用，我们想要执行它们。

我们只是多了一种可供模型使用的工具。

In [7]:
import {
  type BaseMessageLike,
  AIMessage,
  ToolMessage,
} from "@langchain/core/messages";
import { type ToolCall } from "@langchain/core/messages/tool";
import { task } from "@langchain/langgraph";

const toolsByName = Object.fromEntries(tools.map((tool) => [tool.name, tool]));

const callModel = task("callModel", async (messages: BaseMessageLike[]) => {
  const response = await model.bindTools(tools).invoke(messages);
  return response;
});

const callTool = task(
  "callTool",
  async (toolCall: ToolCall): Promise<AIMessage> => {
    const tool = toolsByName[toolCall.name];
    const observation = await tool.invoke(toolCall.args);
    return new ToolMessage({ content: observation, tool_call_id: toolCall.id });
    // 也可以直接将toolCall传递到工具中以返回ToolMessage
    // 返回 tool.invoke(toolCall);
  });

### 定义入口点

我们的[入口点](../../concepts/function_api/#entrypoint) 也与 [ReAct 代理指南](../../how-tos/react-agent-from-scratch-function) 保持不变：

In [8]:
import { entrypoint, addMessages, MemorySaver } from "@langchain/langgraph";

const agent = entrypoint({
  name: "agent",
  checkpointer: new MemorySaver(),
}, async (messages: BaseMessageLike[]) => {
  let currentMessages = messages;
  let llmResponse = await callModel(currentMessages);
  while (true) {
    if (!llmResponse.tool_calls?.length) {
      break;
    }

    // 执行工具
    const toolResults = await Promise.all(
      llmResponse.tool_calls.map((toolCall) => {
        return callTool(toolCall);
      })
    );
    
    // 附加到消息列表
    currentMessages = addMessages(currentMessages, [llmResponse, ...toolResults]);

    // 再次调用模型
    llmResponse = await callModel(currentMessages);
  }

  return llmResponse;
});

### 用法

让我们用一个需要人工帮助的问题来调用我们的模型。我们的问题还需要调用 `getWeather` 工具：

In [9]:
import { BaseMessage, isAIMessage } from "@langchain/core/messages";

const prettyPrintMessage = (message: BaseMessage) => {
  console.log("=".repeat(30), `${message.getType()} message`, "=".repeat(30));
  console.log(message.content);
  if (isAIMessage(message) && message.tool_calls?.length) {
    console.log(JSON.stringify(message.tool_calls, null, 2));
  }
}

const prettyPrintStep = (step: Record<string, any>) => {
  if (step.__metadata__?.cached) {
    return;
  }
  for (const [taskName, update] of Object.entries(step)) {
    const message = update as BaseMessage;
    // 仅打印任务更新
    if (taskName === "agent") continue;
    console.log(`\n${taskName}:`);
    if (taskName === "__interrupt__") {
      console.log(update);
    } else {
      prettyPrintMessage(message);
    }
  }
}

In [11]:
const userMessage = {
  role: "user",
  content: [
    "Can you reach out for human assistance: what should I feed my cat?",
    "Separately, can you check the weather in San Francisco?"
  ].join(" "),
};
console.log(userMessage);

const agentStream = await agent.stream([userMessage], {
  configurable: {
    thread_id: "1",
  }
});

let lastStep;

for await (const step of agentStream) {
  prettyPrintStep(step);
  lastStep = step;
}

{
  role: 'user',
  content: 'Can you reach out for human assistance: what should I feed my cat? Separately, can you check the weather in San Francisco?'
}

callModel:
============================== ai message ==============================

[
  {
    "name": "humanAssistance",
    "args": {
      "query": "What should I feed my cat?"
    },
    "type": "tool_call",
    "id": "call_TwrNq6tGI61cDCJEpj175h7J"
  },
  {
    "name": "getWeather",
    "args": {
      "location": "San Francisco"
    },
    "type": "tool_call",
    "id": "call_fMzUBvc0SpZpXxM2LQLXfbke"
  }
]

callTool:
============================== tool message ==============================
It's sunny!

__interrupt__:
[
  {
    value: { query: 'What should I feed my cat?' },
    when: 'during',
    resumable: true,
    ns: [ 'callTool:2e0c6c40-9541-57ef-a7af-24213a10d5a4' ]
  }
]


请注意，我们生成了两个工具调用，尽管我们的运行被中断，但我们并没有阻止 `get_weather` 工具的执行。

让我们检查一下被中断的地方：

In [12]:
console.log(JSON.stringify(lastStep));

{"__interrupt__":[{"value":{"query":"What should I feed my cat?"},"when":"during","resumable":true,"ns":["callTool:2e0c6c40-9541-57ef-a7af-24213a10d5a4"]}]}


我们可以通过发出[命令](../../concepts/ human_in_the_loop/#the-command-primitive)来恢复执行。请注意，我们在 `Command` 中提供的数据可以根据 `humanAssistance` 的实现根据您的需求进行定制。

In [13]:
import { Command } from "@langchain/langgraph";

const humanResponse = "You should feed your cat a fish.";
const humanCommand = new Command({
  resume: { data: humanResponse },
});

const resumeStream2 = await agent.stream(humanCommand, config);

for await (const step of resumeStream2) {
  prettyPrintStep(step);
}


callTool:
============================== tool message ==============================
You should feed your cat a fish.

callModel:
============================== ai message ==============================
For your cat, it is suggested that you feed it fish. As for the weather in San Francisco, it's currently sunny!


上面，当我们恢复时，我们提供最终的工具消息，允许模型生成其响应。查看 LangSmith 跟踪以查看运行的完整细分：

1. [初始查询追踪](https://smith.langchain.com/public/c007b042-fdd3-49e7-acbe-cadf6969de4b/r)
2.【恢复后追踪】(https://smith.langchain.com/public/1cea310a-13f5-4de9-ae1c-045b8b33015e/r)

**注意：** `interrupt` 函数通过抛出特殊的 `GraphInterrupt` 错误来传播。因此，您应该避免在 `interrupt` 函数周围使用 `try/catch` 块 - 或者如果这样做，请确保在 `catch` 块中再次引发 `GraphInterrupt` 错误。